**Spam Mail Detection using Rocchio and KNN Classifiers**

This notebook implements spam mail detection using two classification algorithms: Rocchio and K-Nearest Neighbors (KNN)

**1. Preprocess the Dataset**

**Step 1:** 

Collect Dataset dataset with a minimum of 50 files of email text. 
Split the dataset into two directories: spam and non_spam.
.

**Step 2:** 

: Preprocessing the Dataset Tokenization: Splitting emails into words. Stop-word removal: Eliminate common words like "and," "the," etc. Lowercasing: Standardize the text by converting everything to lowercase. Stemming/Lemmatization: Reduce words to their root forms (e.g., "running" → "run"). Built-in libraries like nltk or re can be used for preprocessin

Split the Dataset Use 25% of the data for training and the rest for testingg.

In [44]:
#We will load and preprocess the dataset. The preprocessing includes reading files and cleaning the text.

import os
import re
from sklearn.model_selection import train_test_split

# Paths to datasets
spam_path = 'C:\\spam'
non_spam_path = 'C:\\non_spam'

# Load the dataset (Preprocessing: reading files, cleaning text)
def load_data(spam_dir, non_spam_dir):
    emails, labels = [], []
    
    # Reading spam emails
    for filename in os.listdir(spam_dir):
        with open(os.path.join(spam_dir, filename), 'r', encoding='latin-1') as file:
            emails.append(file.read())
            labels.append(1)  # 1 represents spam
    
    # Reading non-spam emails
    for filename in os.listdir(non_spam_dir):
        with open(os.path.join(non_spam_dir, filename), 'r', encoding='latin-1') as file:
            emails.append(file.read())
            labels.append(0)  # 0 represents non-spam
            
    return emails, labels

emails, labels = load_data(spam_path, non_spam_path)

# Cleaning text (remove non-alphanumeric characters, converting to lowercase)
def clean_text(text):
    return re.sub(r'\W+', ' ', text).lower()

emails = [clean_text(email) for email in emails]

# Splitting the dataset (25% for training)
X_train, X_test, y_train, y_test = train_test_split(emails, labels, test_size=0.75, random_state=42)


**2. Compute TF-IDF with Length Normalization Manually**

We will manually compute tf-idf and normalize the vectors.

**Step 2.1:** 

Calculate Term Frequency (TF) For each document, calculate the number of times each word appears (term frequency).

**Step 2.2**

Calculate Inverse Document Frequency (IDF) IDF is calculateds.

IDF = log(N/( 1+Df(t))

 where N is the total number of documents, and DF(t) is the number of documents containing ter

**Step 2.3**

Combine TF and IDF For each word in each document, multiply the term frequency by the inverse document frequency to get the TF-IDF value.

Normalization:normalize the TF-IDF vectors by dividing each component by the length of the vector.

In [43]:
import math
from collections import Counter

# Tokenize and calculate term frequencies
def compute_tf(email):
    words = email.split()
    tf = Counter(words)
    return tf

# Compute inverse document frequency
def compute_idf(emails):
    N = len(emails)
    idf = Counter()
    for email in emails:
        unique_words = set(email.split())
        for word in unique_words:
            idf[word] += 1
    idf = {word: math.log(N / (1 + count)) for word, count in idf.items()}
    return idf

# Compute tf-idf for each email
def compute_tfidf(email, idf):
    tf = compute_tf(email)
    tfidf = {word: tf[word] * idf.get(word, 0) for word in tf}
    return tfidf

idf = compute_idf(X_train)
train_tfidf = [compute_tfidf(email, idf) for email in X_train]

# Length normalization
def normalize_vector(tfidf):
    norm = math.sqrt(sum([value**2 for value in tfidf.values()]))
    if norm == 0:
        return tfidf
    return {word: value / norm for word, value in tfidf.items()}

normalized_train_tfidf = [normalize_vector(tfidf) for tfidf in train_tfidf]

# Display a sample of computed tf-idf
print("Sample TF-IDF Vector:", normalized_train_tfidf[0])


Sample TF-IDF Vector: {'subject': -0.0026971221027534283, 'hourly': 0.30425728751812664, 'gas': 0.08723466314477253, 'deals': 0.17759812356848592, 'fyi': 0.074781417473271, 'if': 0.04301212148212375, 'you': 0.004355226643817182, 'have': 0.011055635415662193, 'any': 0.012930643692439526, 'comments': 0.0858370528889332, 'please': 0.02980369475968716, 'get': 0.02663767836610456, 'them': 0.0359055474186873, 'to': -0.005394244205506857, 'tommy': 0.30425728751812664, 'forwarded': 0.05327535673220912, 'by': 0.007559585750886599, 'brenda': 0.1716741057778664, 'f': 0.05327535673220912, 'herod': 0.1716741057778664, 'hou': 0.19257495642866576, 'ect': 0.33700617375016506, 'on': 0.013065679931451548, '07': 0.149562834946542, '17': 0.1324120008492973, '2000': 0.07913664411708816, '04': 0.04814373910716644, '54': 0.074781417473271, 'pm': 0.03956832205854408, 'enron': 0.11870496617563225, 'technology': 0.05919937452282864, 'from': 0.004355226643817182, 'j': 0.043617331572386266, 'yanowski': 0.10141909

**3. Implement and Train Classifiers**
   
**Rocchio Classifier**

**Step 3.1**

Compute Centroid for Each Class The centroid for each class (spam and non-spam) is computed as the average of the TF-IDF vectors for the documents in that class.

**Step 3.2**

 Classify New Emails Using cosine similarity, classify a new email based on its distance from the centroids of the spam and non-spam classe

compute the centroids for both spam and non_spam.



Cosine Similarity Formula:

Cosine Similarity (𝐴,𝐵)=𝐴⋅𝐵 / ∥𝐴∥∥𝐵∥

In [45]:
# Compute centroid for each class
def compute_centroids(tfidf_data, labels):
    spam_centroid, non_spam_centroid = {}, {}
    spam_count, non_spam_count = 0, 0

    for i, tfidf in enumerate(tfidf_data):
        if labels[i] == 1:  # Spam
            spam_count += 1
            for word, value in tfidf.items():
                spam_centroid[word] = spam_centroid.get(word, 0) + value
        else:  # Non-spam
            non_spam_count += 1
            for word, value in tfidf.items():
                non_spam_centroid[word] = non_spam_centroid.get(word, 0) + value

    # Average the values
    for word in spam_centroid:
        spam_centroid[word] /= spam_count
    for word in non_spam_centroid:
        non_spam_centroid[word] /= non_spam_count
    
    return normalize_vector(spam_centroid), normalize_vector(non_spam_centroid)

spam_centroid, non_spam_centroid = compute_centroids(normalized_train_tfidf, y_train)

# Cosine similarity function
def cosine_similarity(vec1, vec2):
    dot_product = sum([vec1.get(word, 0) * vec2.get(word, 0) for word in vec1])
    norm1 = math.sqrt(sum([value**2 for value in vec1.values()]))
    norm2 = math.sqrt(sum([value**2 for value in vec2.values()]))
    if norm1 == 0 or norm2 == 0:
        return 0
    return dot_product / (norm1 * norm2)

# Classify using Rocchio
def classify_rocchio(email, spam_centroid, non_spam_centroid):
    email_tfidf = normalize_vector(compute_tfidf(email, idf))
    spam_similarity = cosine_similarity(email_tfidf, spam_centroid)
    non_spam_similarity = cosine_similarity(email_tfidf, non_spam_centroid)
    
    return 1 if spam_similarity > non_spam_similarity else 0

# Testing Rocchio classifier
rocchio_predictions = [classify_rocchio(email, spam_centroid, non_spam_centroid) for email in X_test]


**K-Nearest Neighbors (KNN) Classifier**

**Step 4.1:**

Calculate Cosine Similarity For a new email, calculate the cosine similarity between its vector and the vectors of all the training emails

**Step 4.2**

: Determine the Majority Class Select the top k emails with the highest similarity and classify the email as the majority class among the nearest neighbor

store the training vectors and their corresponding labelss.

In [35]:
# Classify based on k-nearest neighbors using cosine similarity
def classify_knn(test_email, train_data, labels, k=3):
    email_tfidf = normalize_vector(compute_tfidf(test_email, idf))
    
    similarities = []
    for i, train_email in enumerate(train_data):
        similarity = cosine_similarity(email_tfidf, train_email)
        similarities.append((similarity, labels[i]))
    
    similarities.sort(reverse=True, key=lambda x: x[0])
    
    # Get the top k neighbors
    top_k = [label for _, label in similarities[:k]]
    
    # Return the most common class
    return 1 if sum(top_k) > len(top_k) / 2 else 0

# Testing KNN classifier
knn_predictions = [classify_knn(email, normalized_train_tfidf, y_train, k=3) for email in X_test]


**4. Evaluate Classifiers**

 Calculate Accuracy ,Compare the predicted labels with the actual labels for both Rocchio and KNN to evaluate the classifiers' performance.


In [36]:
# Accuracy evaluation
def evaluate(predictions, true_labels):
    correct = sum([1 for i in range(len(predictions)) if predictions[i] == true_labels[i]])
    return correct / len(true_labels)

rocchio_accuracy = evaluate(rocchio_predictions, y_test)
knn_accuracy = evaluate(knn_predictions, y_test)

print(f"Rocchio Classifier Accuracy: {rocchio_accuracy:.2f}")
print(f"KNN Classifier Accuracy: {knn_accuracy:.2f}")


Rocchio Classifier Accuracy: 0.89
KNN Classifier Accuracy: 0.84


**5. Discussion**


**Rocchio vs KNN Performance**

**Rocchio Classifier:** May perform well if the spam and non-spam emails are well-separated in the feature space. It is based on centroid similarity which works best when classes are distinct.

**KNN Classifier:** Generally robust to noise and can adapt better to variations in data. Performance depends on the choice of 'k'.

**Effect of 'k' in KNN**

**Small 'k':** The classifier is sensitive to noise and outliers.

**Large 'k':** The classifier may smooth out distinctions between classes but can be less sensitive to local variations.